# Simple Stellar Population (SSP) Feedback with SYGMA

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from NuPyCEE import sygma
from NuPyCEE import omega

### Define functions for this notebook

In [ ]:
%run notebook_functions.py

### Set parameters

In [ ]:
# Parameter dictionary
params = dict()

# AGB and massive stars yields
params["table"] = 'yield_tables/agb_and_massive_stars_C15_LC18_R_mix.txt'
params["imf_yields_range"] = [1.0, 100.0] # [Msun] - Stars in that mass range will eject metals

# Transition mass between AGB and massive star yields
# !!! --> CC SNe will be counted from the transition mass up to imf_yields_range[1]
params["transitionmass"] = 8.0 # [Msun]

# Type of IMF - can be a custom one
params["imf_type"] = 'chabrier'
params["imf_bdys"] = [0.1, 100] # [Msun] - Lower and upper limit of the IMF

# Mass of the stellar population [Msun]
params["mgal"] = 1.0

# SNe Ia
params["sn1a_table"] = 'yield_tables/sn1a_i99_W7.txt'
params["nb_1a_per_m"] = 1.0e-3 # Number of SN Ia per units of Msun formed
params["sn1a_rate"] = 'power_law' # Shape (delay-time distribution) of the SN Ia rate
params["beta_pow"] = -1.0 # Slope of the power law (SN Ia rate)

# Neutron star mergers (NSMs)
params["ns_merger_on"] = True
params["nsmerger_table"] = 'yield_tables/r_process_arnould_2007.txt' # r-process table (based on the solar composition)
params["nsm_dtd_power"] = [3.0e7, 1.0e10, -1.0] # Shape of the NSM rate, t^-1 from 30 Myr to 10 Gyr
params["nb_nsm_per_m"] = 1.5e-5 # Number of NSM per units of Msun formed
params["m_ej_nsm"] = 1.0e-2 # Mass ejected by each NSM 

# Timestep of the output
params["dt"] = 1.0e6
params["special_timesteps"] = 100 # Logarithmic timesteping
# See more timestep options here: 
# https://github.com/NuGrid/NuPyCEE/blob/master/DOC/Capabilities/Timesteps_size_management.ipynb

### Extract the list of available metallicities

In [ ]:
# Run a dummy chemical evolution calculation to access the table info
o_dummy = omega.omega(table=params["table"])
Z_table = o_dummy.Z_table
nb_Z = len(o_dummy.Z_table)

# Print the metallicities
print("\nAvailable metallicities (Z, mass fraction)")
print("  ",Z_table)

### Run SYGMA for all available metallicities

In [ ]:
sygma_instances = []
for i_Z in range(nb_Z):
    sygma_instances.append(sygma.sygma(iniZ=Z_table[i_Z], **params))

### Write feedback table

In [ ]:
# Choose your elements and output file name
elements = ['H', 'He', 'C', 'O', 'Ne', 'Mg', 'Si', 'S', 'Fe', 'Eu', 'Ba']

# Choose the kinetic energy released by sources [erg]
e_per_CC_SN = 1.0e51
e_per_SN_Ia = 1.0e51

# Define the credits for generating the table
prepared_by = "your_name"

# Open the table
enzo_table = open("your_table.txt", "w")

# Write the file header
write_main_header(enzo_table, prepared_by, params)

# For each metallicity 
for i_Z in range(nb_Z):

    # Copy the instance
    inst = sygma_instances[i_Z]
    
    # Write the metallicity and column labels
    write_metallicity_header(enzo_table, inst, elements)
    
    # For each timestep ..
    for i_t in range(inst.nb_timesteps):
        
        # Get the stellar ejecta rate [Msun/yr]
        r_ej_elements, r_ej_tot, r_ej_Z = get_ejecta_rate(inst, i_t, elements)

        # Get the kinetic energy rate [erg/s]
        r_e_CC_SN, r_e_SN_Ia = get_energy_rate(inst, i_t, e_per_CC_SN, e_per_SN_Ia)
        
        # Write time [yr]
        enzo_table.write(fill_with_space('%.4e'%inst.history.age[i_t]))
        
        # Write the mass ejection rates [Msun/yr]
        for i_elem in range(len(r_ej_elements)):
            enzo_table.write(fill_with_space('%.4e'%r_ej_elements[i_elem]))
        enzo_table.write(fill_with_space('%.4e'%r_ej_tot))
        enzo_table.write(fill_with_space('%.4e'%r_ej_Z))
        
        # Write energy rate [erg/s]
        enzo_table.write(fill_with_space('%.4e'%r_e_CC_SN))
        enzo_table.write(fill_with_space('%.4e'%r_e_SN_Ia))
        
        # Prepare for the next time entry
        enzo_table.write("\n")
        
# Close the table
enzo_table.close()